<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/notebooks/tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init - import library - download data

In [1]:
!rm -rf kltn
!rm -rf sample_data
!git clone -b dev https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 1541, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 1541 (delta 59), reused 46 (delta 21), pack-reused 1444 (from 1)
Receiving objects: 100% (1541/1541), 16.27 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (1042/1042), done.


In [2]:
from kltn.init.install_dependencies import install_packages, load_config
install_packages("./kltn")
config = load_config("./kltn/parameters.yaml")

In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import json

from kltn import *
from kltn.datasets.get_datasets import get_datasets_lits, get_liver_mask
from kltn.utils.logger import init_logger
from kltn.utils.train_val_epochs import  trainer_stage2
from kltn.utils.metrics import  DiceMetric, TverskyLossWSigmoid, DiceLossWSigmoid
from kltn.models.model_factory import ModelFactory
from kltn.utils.visualize_results import visualize_results_stage_2

In [4]:
drive.flush_and_unmount()
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
LOGGER = init_logger(config["log_path"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
from kltn.datasets.prepare_data import prepare_dataset_LiTS
prepare_dataset_LiTS(dir_name=config["source_folder_lits"])

0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 1) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
LITS17 (udp://tracker.opentrackr.org:1337/announce)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
0.51% complete (down: 8038.2 kB/s up: 229.7 kB/s peers: 9) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://ipv6.academictorrents.com/announce.php

## Model

In [6]:
weight_path = "/content/drive/MyDrive/KLTN/code/weight/best_metric_model_UNet3DWResNeXtCoT_liver.pth"
model_stage_1 = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)
model_stage_1.load_state_dict(torch.load(weight_path, map_location=device))

<All keys matched successfully>

In [7]:
model = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)

In [8]:
liver_masks = get_liver_mask(config["source_folder_lits"], model_stage_1, device)

## Load data

In [9]:
full_train_dataset, val_dataset = get_datasets_lits(source_folder=config["source_folder_lits"], seed=123, fold_number=2, mode="tumor", liver_masks = liver_masks)
print(len(full_train_dataset), len(val_dataset))

105 26


In [10]:
train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=1, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=1)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import torch

def visualize_from_loader(image_tensor, label_tensor):
    """
    Visualize 3D volume (from DataLoader) using Max Intensity Projection (MIP)

    Args:
        image_tensor: Tensor, shape [1, 1, D, H, W]
        label_tensor: Tensor, shape [1, 1, D, H, W]

    Notes:
        - Converts to numpy
        - Projects to 2D using max projection along depth
    """

    # Convert to numpy, squeeze to shape [D, H, W]
    image = image_tensor.detach().cpu().numpy().squeeze()
    label = label_tensor.detach().cpu().numpy().squeeze()

    # Max projection along depth
    image_2d = np.max(image, axis=0)
    label_2d = np.max(label, axis=0)

    # Color map for tumor mask
    cmap = mcolors.ListedColormap(["black", "yellow"])

    # Plotting
    num_cols = 2
    plt.figure(figsize=(5 * num_cols, 5))

    plt.subplot(1, num_cols, 1)
    plt.title("Input Image (MIP)")
    plt.imshow(image_2d, cmap='gray')
    plt.axis("off")

    plt.subplot(1, num_cols, 2)
    plt.title("Ground Truth (Tumor MIP)")
    plt.imshow(image_2d, cmap='gray')
    plt.imshow(label_2d, cmap=cmap, alpha=0.5)
    plt.axis("off")


    plt.tight_layout()
    plt.show()


In [ ]:
i = 0
for train_data in train_loader:
    i+= 1
    image = train_data["image"]
    label = train_data["label"]

    visualize_from_loader(image, label)
    if i == 10:
      break

## Training

In [ ]:
criterion = TverskyLossWSigmoid().to(device)

dice_acc = DiceMetric(include_background=False, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=float(config["learning_rate_train"]), weight_decay=float(config["weight_decay"]))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(config["max_epochs"]), eta_min = float(config["eta_min"]))

In [ ]:
LOGGER.info("[TRAINER] Start TRAIN process...")

(
    val_acc_max,
    best_epoch,
    dices_tumor,
    loss_epochs,
    trains_epoch,
    ious_tumor,
    precisions_tumor,
    recalls_tumor,
    time_tmp
) = trainer_stage2(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=criterion,
    acc_func=dice_acc,
    scheduler=scheduler,
    batch_size=config["batch_size"],
    max_epochs = config["max_epochs"],
    start_epoch = config["start_epoch"],
    val_every=config["val_every"],
    path_save_model=config["path_save_model_state"],
    logger=LOGGER,
    save_model=True,
    post_fix="tumor"
)

[TRAINER] Start TRAIN process...
INFO:kltn.utils.logger:[TRAINER] Start TRAIN process...

==============================Training epoch 1==============================
INFO:kltn.utils.logger:
==============================Training epoch 1==============================


Epoch 1/100 1/105 loss: 1.1628 time 6.78s
Epoch 1/100 2/105 loss: 1.1043 time 9.02s
Epoch 1/100 3/105 loss: 1.0814 time 3.06s
Epoch 1/100 4/105 loss: 1.0991 time 4.92s
Epoch 1/100 5/105 loss: 1.0960 time 1.50s
Epoch 1/100 6/105 loss: 1.1095 time 5.57s
Epoch 1/100 7/105 loss: 1.1200 time 1.40s
Epoch 1/100 8/105 loss: 1.1189 time 9.32s
Epoch 1/100 9/105 loss: 1.1206 time 4.22s
Epoch 1/100 10/105 loss: 1.1203 time 1.56s
Epoch 1/100 11/105 loss: 1.1140 time 1.58s
Epoch 1/100 12/105 loss: 1.1144 time 3.42s
Epoch 1/100 13/105 loss: 1.1062 time 23.71s
Epoch 1/100 14/105 loss: 1.1082 time 5.43s
Epoch 1/100 15/105 loss: 1.1081 time 3.98s
Epoch 1/100 16/105 loss: 1.1091 time 1.57s
Epoch 1/100 17/105 loss: 1.1093 time 1.62s
Epoch 1/100 18/105 loss: 1.1110 time 4.95s
Epoch 1/100 19/105 loss: 1.1110 time 1.56s
Epoch 1/100 20/105 loss: 1.1103 time 3.48s
Epoch 1/100 21/105 loss: 1.1105 time 7.15s
Epoch 1/100 22/105 loss: 1.1098 time 8.86s
Epoch 1/100 23/105 loss: 1.1081 time 1.38s
Epoch 1/100 24/105 

Final training epochs: 1/100 ---[loss: 1.0087] ---[time 511.28s]
INFO:kltn.utils.logger:Final training epochs: 1/100 ---[loss: 1.0087] ---[time 511.28s]

********************Epoch 1 Validation********************
INFO:kltn.utils.logger:
********************Epoch 1 Validation********************


Epoch 1/100 105/105 loss: 1.0087 time 6.40s


Val 1/100 1/26, Dice_Tumor: 0.000581, time 2.63s
INFO:kltn.utils.logger:Val 1/100 1/26, Dice_Tumor: 0.000581, time 2.63s
Val 1/100 2/26, Dice_Tumor: 0.031107, time 6.39s
INFO:kltn.utils.logger:Val 1/100 2/26, Dice_Tumor: 0.031107, time 6.39s
Val 1/100 3/26, Dice_Tumor: 0.006166, time 5.99s
INFO:kltn.utils.logger:Val 1/100 3/26, Dice_Tumor: 0.006166, time 5.99s
Val 1/100 4/26, Dice_Tumor: 0.000000, time 5.84s
INFO:kltn.utils.logger:Val 1/100 4/26, Dice_Tumor: 0.000000, time 5.84s
Val 1/100 5/26, Dice_Tumor: 0.048118, time 7.39s
INFO:kltn.utils.logger:Val 1/100 5/26, Dice_Tumor: 0.048118, time 7.39s
Val 1/100 6/26, Dice_Tumor: 0.006663, time 2.27s
INFO:kltn.utils.logger:Val 1/100 6/26, Dice_Tumor: 0.006663, time 2.27s
Val 1/100 7/26, Dice_Tumor: 0.012155, time 7.17s
INFO:kltn.utils.logger:Val 1/100 7/26, Dice_Tumor: 0.012155, time 7.17s
Val 1/100 8/26, Dice_Tumor: 0.000162, time 6.36s
INFO:kltn.utils.logger:Val 1/100 8/26, Dice_Tumor: 0.000162, time 6.36s
Val 1/100 9/26, Dice_Tumor: 0.00

...

Epoch 85/100 1/105 loss: 0.3130 time 7.26s
Epoch 85/100 2/105 loss: 0.2347 time 2.10s
Epoch 85/100 3/105 loss: 0.2019 time 1.53s
Epoch 85/100 4/105 loss: 0.2615 time 1.18s
Epoch 85/100 5/105 loss: 0.2594 time 6.75s
Epoch 85/100 6/105 loss: 0.2644 time 1.91s
Epoch 85/100 7/105 loss: 0.3279 time 3.19s
Epoch 85/100 8/105 loss: 0.3364 time 5.93s
Epoch 85/100 9/105 loss: 0.3763 time 3.87s
Epoch 85/100 10/105 loss: 0.3564 time 10.17s
Epoch 85/100 11/105 loss: 0.3491 time 4.26s
Epoch 85/100 12/105 loss: 0.3373 time 8.06s
Epoch 85/100 13/105 loss: 0.3194 time 2.65s
Epoch 85/100 14/105 loss: 0.3142 time 9.42s
Epoch 85/100 15/105 loss: 0.3400 time 2.41s
Epoch 85/100 16/105 loss: 0.3336 time 6.47s
Epoch 85/100 17/105 loss: 0.3548 time 1.23s
Epoch 85/100 18/105 loss: 0.3552 time 1.91s
Epoch 85/100 19/105 loss: 0.3420 time 8.00s
Epoch 85/100 20/105 loss: 0.3309 time 5.48s
Epoch 85/100 21/105 loss: 0.3215 time 8.22s
Epoch 85/100 22/105 loss: 0.3134 time 3.63s
Epoch 85/100 23/105 loss: 0.3078 time 1.

Final training epochs: 85/100 ---[loss: 0.3194] ---[time 537.56s]
INFO:kltn.utils.logger:Final training epochs: 85/100 ---[loss: 0.3194] ---[time 537.56s]

********************Epoch 85 Validation********************
INFO:kltn.utils.logger:
********************Epoch 85 Validation********************


Epoch 85/100 105/105 loss: 0.3194 time 9.46s


Val 85/100 1/26, Dice_Tumor: 0.000000, time 2.75s
INFO:kltn.utils.logger:Val 85/100 1/26, Dice_Tumor: 0.000000, time 2.75s
Val 85/100 2/26, Dice_Tumor: 0.115724, time 6.43s
INFO:kltn.utils.logger:Val 85/100 2/26, Dice_Tumor: 0.115724, time 6.43s
Val 85/100 3/26, Dice_Tumor: 0.354667, time 6.60s
INFO:kltn.utils.logger:Val 85/100 3/26, Dice_Tumor: 0.354667, time 6.60s
Val 85/100 4/26, Dice_Tumor: 0.000000, time 5.66s
INFO:kltn.utils.logger:Val 85/100 4/26, Dice_Tumor: 0.000000, time 5.66s
Val 85/100 5/26, Dice_Tumor: 0.051936, time 6.55s
INFO:kltn.utils.logger:Val 85/100 5/26, Dice_Tumor: 0.051936, time 6.55s
Val 85/100 6/26, Dice_Tumor: 0.329462, time 2.49s
INFO:kltn.utils.logger:Val 85/100 6/26, Dice_Tumor: 0.329462, time 2.49s
Val 85/100 7/26, Dice_Tumor: 0.066593, time 8.19s
INFO:kltn.utils.logger:Val 85/100 7/26, Dice_Tumor: 0.066593, time 8.19s
Val 85/100 8/26, Dice_Tumor: 0.000000, time 6.60s
INFO:kltn.utils.logger:Val 85/100 8/26, Dice_Tumor: 0.000000, time 6.60s
Val 85/100 9/26,

Epoch 86/100 1/105 loss: 0.1914 time 2.10s
Epoch 86/100 2/105 loss: 0.2873 time 6.46s
Epoch 86/100 3/105 loss: 0.4251 time 7.80s
Epoch 86/100 4/105 loss: 0.3705 time 5.89s
Epoch 86/100 5/105 loss: 0.3687 time 6.87s
Epoch 86/100 6/105 loss: 0.3516 time 9.48s
Epoch 86/100 7/105 loss: 0.3294 time 6.37s
Epoch 86/100 8/105 loss: 0.3081 time 2.49s
Epoch 86/100 9/105 loss: 0.2856 time 8.84s
Epoch 86/100 10/105 loss: 0.3073 time 2.50s
Epoch 86/100 11/105 loss: 0.2930 time 3.19s
Epoch 86/100 12/105 loss: 0.3271 time 3.59s
Epoch 86/100 13/105 loss: 0.3194 time 5.53s
Epoch 86/100 14/105 loss: 0.3118 time 1.91s
Epoch 86/100 15/105 loss: 0.2944 time 2.03s
Epoch 86/100 16/105 loss: 0.2941 time 11.08s
Epoch 86/100 17/105 loss: 0.2836 time 7.95s
Epoch 86/100 18/105 loss: 0.2738 time 3.49s
Epoch 86/100 19/105 loss: 0.2657 time 2.43s
Epoch 86/100 20/105 loss: 0.2813 time 4.54s
Epoch 86/100 21/105 loss: 0.2770 time 3.53s
Epoch 86/100 22/105 loss: 0.2963 time 6.91s
Epoch 86/100 23/105 loss: 0.2946 time 4.

In [ ]:
import json
path_save_result = f"/content/drive/MyDrive/KLTN/code/result_model_{model.__class__.__name__}_tumor.json"

results = {
    "dice_tumor": [float(x) for x in np.array(dices_tumor, dtype=np.float32)],
    "loss": [float(x) for x in np.array(loss_epochs, dtype= np.float32)],
    "iou_tumor": [float(x) for x in np.array(ious_tumor, dtype=np.float32)],
    "precision_tumor": [float(x) for x in np.array(precisions_tumor, dtype=np.float32)],
    "recall_tumor": [float(x) for x in np.array(recalls_tumor, dtype=np.float32)],
    "best_epoch": best_epoch,
    "time_train": time_tmp
}
with open(path_save_result, "w") as f:
    json.dump(results, f, indent=4)

## Víualize result

In [ ]:
import glob
print(glob.glob(f"{config['path_save_model_state']}/best_metric_model*"))

In [ ]:
num_images = len(val_loader)
weight_path = "/content/drive/MyDrive/KLTN/code/weight/best_metric_model_UNet3DWResNeXtCoT_liver.pth"
visualize_results_stage_2(model, val_loader, weight_path, num_images, device)

## Result Metrics

In [6]:
def load_metrics(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

In [7]:
import glob
print(glob.glob("/content/drive/MyDrive/KLTN/code/result_model_*.json"))

['/content/drive/MyDrive/KLTN/code/result_model_UNet3D.json', '/content/drive/MyDrive/KLTN/code/result_model_UNet3DWCoT.json', '/content/drive/MyDrive/KLTN/code/result_model_UNet3DWCoTDA.json', '/content/drive/MyDrive/KLTN/code/result_model_UNet3DWResNeXtCoT.json', '/content/drive/MyDrive/KLTN/code/result_model_UNet3DPretrained_pretrained.json', '/content/drive/MyDrive/KLTN/code/result_model_UNet3DWConvNeXtV2CoT.json', '/content/drive/MyDrive/KLTN/code/result_model_UNet3DWResNeXtCoT_liver.json', '/content/drive/MyDrive/KLTN/code/result_model_UNet3DWResNeXtCoT_tumor.json']


In [8]:
model_result = load_metrics("/content/drive/MyDrive/KLTN/code/result_model_UNet3DWResNeXtCoT_tumor.json")

In [9]:
epochs = np.arange(1, len(model_result["dice_tumor"]) + 1)
plt.figure(figsize=(14, 7))

# dice
plt.subplot(1, 2, 1)
plt.plot(epochs, model_result["dice_tumor"], label="Dice", color='blue')
plt.title("Dice Coefficient")
plt.xlabel("Epochs")
plt.ylabel("Dice Coefficient")
plt.legend()
plt.grid()

# loss
plt.subplot(1, 2, 2)
plt.plot(epochs, model_result["loss"], label="Loss", color='red')
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()